In [8]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib as plt
import pandas as pd
# jesse. jesse we have to train. 

In [30]:
def refactor(data, type):
     if type == 'all':
          features = data.filter(regex = 'score')
     else:
          features = data.filter(regex = type + 'score')
     labels = data.filter(regex = 'Amphet')
     
     labels['Amphet'] = labels['Amphet'].map({'CL0': 0, 
                                            'CL1': 0, 
                                            'CL2': 1, 
                                            'CL3': 1, 
                                            'CL4': 1, 
                                            'CL5': 1,
                                            'CL6': 1})

     return features, labels

In [40]:
raw_drugs = pd.read_csv(r'tailored_drug_dataset.csv')

In [41]:
def trainModel(dataset, bigFive, test_size):
    X, y = refactor(dataset, bigFive)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)

    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    score_train = (lr.score(X_train, y_train))
    score_test = (lr.score(X_test, y_test))

    return score_train, score_test

In [47]:
testing = ['all', 'O', 'C', 'E', 'A', 'N']
for i in testing:
    print(trainModel(raw_drugs, i, 0.2))


(0.6584880636604774, 0.713527851458886)
(0.6438992042440318, 0.6578249336870027)
(0.6432360742705571, 0.6551724137931034)
(0.639920424403183, 0.6392572944297082)
(0.6432360742705571, 0.6445623342175066)
(0.6412466843501327, 0.623342175066313)


/var/folders/gh/96n2f5tx5yv8ywqs48jsyz7m0000gn/T/ipykernel_43957/221857216.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['Amphet'] = labels['Amphet'].map({'CL0': 0,
/Users/karenli/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/gh/96n2f5tx5yv8ywqs48jsyz7m0000gn/T/ipykernel_43957/221857216.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand